# Practica 2 - Natural Language processing

## Librerías

In [1]:
import numpy as np

import tensorflow as tf

import pandas as pd
from collections import Counter
import re

from tensorflow import keras


import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.utils import to_categorical

from tensorflow.keras import Model, Input, layers
from tensorflow.keras.layers import Embedding, Dot, Reshape, Dense

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams

from keras.callbacks import TensorBoard

import re
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer

import numpy as np
from tensorflow.keras.preprocessing.sequence import skipgrams

## Funciones

## Descarga y proceso de datos

### Carga de datos

In [2]:
import os
import pandas as pd

# 1. Directorio portable a P2_NaturalLanguageProcessing/data
cwd      = os.getcwd()
data_dir = os.path.join(cwd, 'data')

# 2. Rutas a los CSV
train_path = os.path.join(data_dir, 'train.csv')
test_path  = os.path.join(data_dir, 'test.csv')

# 3. Lista de codificaciones a probar
encodings = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']

def load_csv(path, enc_list):
    for enc in enc_list:
        try:
            df = pd.read_csv(path, encoding=enc)
            print(f"Cargado '{os.path.basename(path)}' con encoding {enc}")
            return df, enc
        except UnicodeDecodeError:
            print(f"Fallo con encoding {enc}, probando siguiente...")
    raise ValueError(f"No se pudo decodificar {path} con las codificaciones {enc_list}")

# 4. Cargar ambos conjuntos usando la misma codificación
train_df, used_enc = load_csv(train_path, encodings)
test_df, _      = load_csv(test_path, [used_enc])

# 5. Verificación
print(f"Train shape: {train_df.shape}")
print(f"Test  shape: {test_df.shape}")


Fallo con encoding utf-8, probando siguiente...
Cargado 'train.csv' con encoding latin1
Cargado 'test.csv' con encoding latin1
Train shape: (27481, 10)
Test  shape: (4815, 9)


### Exploración y limpieza de datos

In [3]:
# 5. Juntar los DataFrames para facilitar el preprocesado
data_df = pd.concat([train_df, test_df], ignore_index=True)      # concatenación[2]

# 6. Inspeccionar estructura básica
print(data_df.info())
print(data_df.head())

# 7. Contar palabras únicas en la columna 'text' para estimar VOCAB_SIZE
all_text = ' '.join(data_df['text'].astype(str)).lower()
words = re.findall(r'\b\w+\b', all_text)
word_counts = Counter(words)
unique_words = len(word_counts)

print(f"Total de muestras: {data_df.shape[0]}")
print(f"Palabras únicas encontradas: {unique_words}")
print("Top 10 palabras más frecuentes:", word_counts.most_common(10))

# 8. Definir parámetros para TensorFlow/Keras
BUFFER_SIZE = 32768      # chivo mayor al dataset para buen shuffle 2^15
BATCH_SIZE = 128        # potencia de 2 adecuada para GPU
VOCAB_SIZE = 8192      # tamaño del vocabulario[4]
                 
print("BUFFER_SIZE =", BUFFER_SIZE)
print("BATCH_SIZE =", BATCH_SIZE)
print("VOCAB_SIZE =", VOCAB_SIZE)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32296 entries, 0 to 32295
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   textID            31015 non-null  object 
 1   text              31014 non-null  object 
 2   selected_text     27480 non-null  object 
 3   sentiment         31015 non-null  object 
 4   Time of Tweet     31015 non-null  object 
 5   Age of User       31015 non-null  object 
 6   Country           31015 non-null  object 
 7   Population -2020  31015 non-null  float64
 8   Land Area (Km²)   31015 non-null  float64
 9   Density (P/Km²)   31015 non-null  float64
dtypes: float64(3), object(7)
memory usage: 2.5+ MB
None
       textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...  

### Preparación de datos

#### Creación del corpus

In [4]:
# 1. Seleccionar las 4 096 palabras más frecuentes
most_common_words = {w for w, _ in word_counts.most_common(VOCAB_SIZE)}

# 2. Filtrar cada texto para quedarnos solo con tokens en el top 4 096
corpus_filtered = []
for text in data_df['text'].dropna().astype(str):
    tokens = re.findall(r'\b\w+\b', text.lower())
    filtered_tokens = [t for t in tokens if t in most_common_words]
    corpus_filtered.append(" ".join(filtered_tokens))

# 3. Tokenizar el corpus filtrado
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(corpus_filtered)

# 4. Convertir textos a secuencias de índices
sequences = tokenizer.texts_to_sequences(corpus_filtered)

# 5. Diccionarios de mapeo y tamaño final de vocabulario
word2idx = tokenizer.word_index
idx2word = {i: w for w, i in word2idx.items()}
vocab_size = min(len(word2idx) + 1, VOCAB_SIZE)

# 6. Mostrar resultados de prueba
print("Primeros 5 textos filtrados:", corpus_filtered[:5])
print("Primeras 5 secuencias:", sequences[:5])
print("Tamaño de vocabulario efectivo:", vocab_size)


Primeros 5 textos filtrados: ['i d have responded if i were going', 'sooo sad i will miss you here in san diego', 'my boss is me', 'what interview leave me alone', 'sons of why couldn t they put them on the releases we already bought']
Primeras 5 secuencias: [[1, 163, 19, 7648, 71, 1, 151, 49], [421, 117, 1, 63, 94, 7, 91, 10, 1447, 2230], [5, 1410, 9, 16], [51, 1193, 350, 16, 495], [4254, 13, 118, 472, 14, 72, 332, 131, 17, 3, 7649, 50, 210, 569]]
Tamaño de vocabulario efectivo: 8152


#### Generación de los pares de entrenamiento

In [5]:
# Parámetros generales
# vocab_size_eff    = vocab_size    # p.ej. 8192
negative_samples  = 0.0
seed_value        = 42

# Ventanas para generar pares
window_sizes = [2, 4]

# Contenedor de pares por tamaño de ventana
pairs_by_window = {}

for w in window_sizes:
    pairs = []
    for seq in sequences:
        if len(seq) < 2:
            continue
        # Generar solo pares positivos con skipgrams
        pos_pairs, _ = skipgrams(
            sequence=seq,
            vocabulary_size=vocab_size,
            window_size=w,
            negative_samples=negative_samples,
            shuffle=True,
            seed=seed_value
        )
        pairs.extend(pos_pairs)

    # Convertir a arrays target/context
    if pairs:
        t, c = zip(*pairs)
        targets  = np.array(t, dtype='int32')
        contexts = np.array(c, dtype='int32')
    else:
        targets  = np.zeros((0,), dtype='int32')
        contexts = np.zeros((0,), dtype='int32')

    pairs_by_window[w] = {
        'pairs':   pairs,
        'targets': targets,
        'contexts':contexts
    }
    print(f"Window={w}: {len(pairs)} pares generados")

# Ejemplo de acceso:
# pairs_by_window[2]['targets'], pairs_by_window[2]['contexts']
# pairs_by_window[4]['targets'], pairs_by_window[4]['contexts']



Window=2: 1384586 pares generados
Window=4: 2530442 pares generados


#### Integer-encoding
Un poco de teoría de los dos encoders:

- Integer encoding: X_target y X_context son vectores de enteros donde cada valor es el índice de palabra en el vocabulario. Es lo adecuado cuando se usa una capa Embedding, ya que Keras transforma internamente cada índice en su vector embebido.

- One-hot encoding: cada índice se convierte en un vector binario de longitud vocab_size con un único 1 en la posición correspondiente al índice. Ocupa mucha más memoria y rara vez es necesario si tu primer bloque es Embedding.

In [6]:
import numpy as np

# Suponemos que ya existe:
#  - pairs_by_window: dict con keys 2 y 4, cada uno contiene 'targets' y 'contexts'
#  - vocab_size: tamaño efectivo del vocabulario (p.ej. 8192)

datasets = {}

for w, data in pairs_by_window.items():
    # 1. Extraer targets y contexts
    targets  = data['targets']    # array shape (n_pairs,)
    contexts = data['contexts']   # array shape (n_pairs,)
    
    # 2. Crear X_train como representación dispersa (enteros)
    X_target  = targets
    X_context = contexts
    X_train   = [X_target, X_context]
    
    # 3. Etiquetas positivas
    y_train = np.ones((len(targets),), dtype='int32')
    
    # 4. Almacenar en el nuevo dict
    datasets[w] = {
        'X_train': X_train,
        'y_train': y_train
    }
    
    # 5. Mostrar shapes de comprobación
    print(f"--- Ventana = {w} ---")
    print("Número de pares:", len(targets))
    print("X_target shape:",  X_target.shape)
    print("X_context shape:", X_context.shape)
    print("y_train shape:",    y_train.shape)
    print()

# Acceso posterior:
# datasets[2]['X_train'], datasets[2]['y_train']
# datasets[4]['X_train'], datasets[4]['y_train']

--- Ventana = 2 ---
Número de pares: 1384586
X_target shape: (1384586,)
X_context shape: (1384586,)
y_train shape: (1384586,)

--- Ventana = 4 ---
Número de pares: 2530442
X_target shape: (2530442,)
X_context shape: (2530442,)
y_train shape: (2530442,)



## Arquitectura y ejecucción

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Reshape
from tensorflow.keras.optimizers import Adam

def build_skipgram_model(vocab_size, 
                         embedding_dim, 
                         learning_rate=0.025,
                         beta_1=0.9, 
                         beta_2=0.999):
    """
    Devuelve un modelo Skip-Gram compilado que:
     - Recibe dos inputs enteros (target y context, cada uno shape=())
     - Usa una capa Embedding compartida de tamaño (vocab_size, embedding_dim)
     - Calcula el dot-product y pasa por sigmoide
     - Usa binary_crossentropy + Adam
    """
    # 1) capas de entrada (índice único por ejemplo)
    target_input  = Input(shape=(), dtype='int32', name='target_input')
    context_input = Input(shape=(), dtype='int32', name='context_input')
    
    # 2) capa Embedding compartida
    embed = Embedding(input_dim=vocab_size,
                      output_dim=embedding_dim,
                      input_length=1,
                      name='shared_embedding',
                      embeddings_initializer='glorot_uniform',
                      trainable=True)
    
    # 3) obtenemos los vectores embebidos y aplanamos
    target_vec  = Reshape((embedding_dim,)) (embed(target_input))
    context_vec = Reshape((embedding_dim,)) (embed(context_input))
    
    # 4) puntuación = dot-product
    dot_prod = Dot(axes=1, normalize=False, name='dot_product')(
        [target_vec, context_vec]
    )
    
    # 5) salida sigmoide para predecir par positivo/negativo
    output = Reshape((1,))(dot_prod)
    output = tf.keras.activations.sigmoid(output)
    
    # 6) construir y compilar modelo
    model = Model(inputs=[target_input, context_input], outputs=output, name=f"skipgram_{embedding_dim}")
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2),
        metrics=['accuracy']
    )
    return model

# --------------------------------------------------------------
# Ejemplo de entrenamiento de los 6 modelos (3 dims × 2 ventanas)
# --------------------------------------------------------------
embedding_dims = [45, 312, 752]
window_sizes   = [2, 4]
histories      = {}

for w in window_sizes:
    X_target, X_context = datasets[w]['X_train']
    y_train             = datasets[w]['y_train']
    
    for ed in embedding_dims:
        print(f"\nEntrenando Skip-Gram: window={w}, embed_dim={ed}")
        
        # 1) construir
        model = build_skipgram_model(vocab_size=vocab_size, 
                                     embedding_dim=ed,
                                     learning_rate=0.025)
        
        # 2) entrenar
        history = model.fit(
            x=[X_target, X_context],
            y=y_train,
            batch_size=BATCH_SIZE,
            epochs=10,                # ajusta número de épocas
            verbose=1
        )
        
        # 3) guardar historia y modelo
        histories[(w, ed)] = history

# Ahora `histories[(w,ed)].history['loss']` contiene la curva de pérdida



Entrenando Skip-Gram: window=2, embed_dim=45
Epoch 1/10


c:\Users\jbarc\Documents\Deep learning\UFVDeepLearning\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


10818/10818 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9844 - loss: 0.0267
Epoch 2/10
10818/10818 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 1.0000 - loss: 3.4643e-07
Epoch 3/10
10818/10818 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 1.0000 - loss: 4.0106e-08
Epoch 4/10
10818/10818 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 1.0000 - loss: 2.0869e-08
Epoch 5/10
10818/10818 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 1.0000 - loss: 1.4148e-08
Epoch 6/10
10818/10818 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 1.0000 - loss: 1.0934e-08
Epoch 7/10
10818/10818 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 1.0000 - loss: 8.7105e-09
Epoch 8/10
10818/10818 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 1.0000 - loss: 7.3767e-09
Epoch 9/10
10818/10818 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 1.0000 - loss: 6.3736e-09
Epoch 10/10
10818/10818 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 1.0000 - loss: 5.6439e-09

Entrenando Skip-Gram: window=2, embed_dim=312
Epoch 1/10
10818/

KeyboardInterrupt: 

[]
